In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from pymongo import MongoClient
import pydicom as pyd
import matplotlib.pyplot as plt
import pandas as pd
from skimage.transform import resize
from skimage import measure
import random
from random import shuffle
import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import keras_preprocessing.image as KPImage
from keras.applications.vgg19 import VGG19
from PIL import Image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Input, Conv2D, multiply, LocallyConnected2D, Lambda, AvgPool2D
from keras.models import Model
from keras.optimizers import Adam
from keras import layers

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = MongoClient('localhost', 27017)
db = client['rsna-xray']
col = db['patient']

In [3]:
#Getting the next batch of X and y
def next_batch(batchsize, documents, size = (256, 256), shuffle = True,
               classes = ["No Lung Opacity / Not Normal", "Lung Opacity", "Normal"], stacked = False):
    """
    This function is meant to return the image array and the encoded labels for the 
    training of the neural network. 
    
    batchsize - int - how many instances you would want retreived
    
    documents - list - the list of documents you want the batch to come from
    
    size - tuple or list - the size of the output array want
    
    classes - Only change when not using the same three classes, keeps the labels the same
    """
    if shuffle:
        #randomly shuffles the documents
        idx = np.arange(0, len(documents))
        np.random.shuffle(idx)
        #Getting that number if indices
        idx = idx[:batchsize]
        #Creates a list for that batch
        batch = [documents[i] for i in idx]
        #A numpy matrix of the images
    else:
        batch = documents[:batchsize]
    
    #numpy array for the classes
    y = np.zeros([batchsize, 3])
    #Goes through the batch
    if stacked:
        X = np.zeros([batchsize, size[0], size[1], 3])
        for i, d in enumerate(batch):
            #reads the file
            f = pyd.read_file(d["path"])
            #Inserts the reshaped image in the data frame
            X[i] = resize(image=np.stack([f.pixel_array] * 3, axis=2), output_shape=size)
            #One hot encodes the class
            y[i, classes.index(d["class"])] = 1
    else:
        X = np.zeros([batchsize, size[0], size[1]])
        for i, d in enumerate(batch):
            #reads the file
            f = pyd.read_file(d["path"])
            #Inserts the reshaped image in the data frame
            X[i] = resize(image=f.pixel_array, output_shape=size)
            #One hot encodes the class
            y[i, classes.index(d["class"])] = 1        
    return X, y

In [4]:
normal = list(col.find({"class":"Normal"}))
lung_opacity = list(col.find({"class":"Lung Opacity"}))
not_normal = list(col.find({"class":"No Lung Opacity / Not Normal"}))

In [5]:
np.random.seed(1000)
#Shuffling the documents
np.random.shuffle(normal)
np.random.shuffle(not_normal)
np.random.shuffle(lung_opacity)
#Getting first 4000 of the shuffled list
normal_train = normal[:4000]
opacity_train = lung_opacity[:4000]
not_normal_train = not_normal[:4000]

#getting 4001 to 6000 of each list
normal_test = normal[4000:6000]
opacity_test = lung_opacity[4000:6000]
not_normal_test = not_normal[4000:6000]

#Setting up training and testing list
train = normal_train + not_normal_train + opacity_train
np.random.shuffle(train)
test = normal_test + not_normal_test + opacity_test
np.random.shuffle(test)

# VGG19

In [6]:
#Getting the pretained model with the input size of 256, 256, 3
#Not including the dense layers
#Not retraining the CNN Part, only the dense layers
pretrained_VGG19 = VGG19(input_shape =  (256, 256, 3), 
                              include_top = False, weights = 'imagenet')
pretrained_VGG19.trainable = False

In [7]:
#Shape out of the pretrained
pretrained_features = Input(pretrained_VGG19.get_output_shape_at(0)[1:], name = 'feature_input')
#Normalizing the features
norm_features = BatchNormalization()(pretrained_features)
#Global pooling
global_pooling = GlobalAveragePooling2D()(norm_features)
#Dropout
pooling_dr = Dropout(0.25)(global_pooling)
#Fully connected and dropout
dr_steps = Dropout(0.25)(Dense(128, activation = 'linear', use_bias=False)(pooling_dr))
#Batch Normalization x2
dr_steps = BatchNormalization()(dr_steps)
#Activation on normalization
dr_steps = layers.LeakyReLU(0.1)(dr_steps)
#Prediction
out_layer = Dense(3, activation = 'softmax')(dr_steps)
#Output model
out_model = Model(inputs = [pretrained_features], 
                   outputs = [out_layer], name = 'trained_model')

out_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_input (InputLayer)   (None, 8, 8, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65536     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
__________

In [8]:
from keras.models import Sequential
from keras.optimizers import Adam
pneumonia_vgg19 = Sequential()
pretrained_VGG19.trainable = False
pneumonia_vgg19.add(pretrained_VGG19)
pneumonia_vgg19.add(out_model)
pneumonia_vgg19.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy',
                           metrics = ['categorical_accuracy'])
pneumonia_vgg19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
trained_model (Model)        (None, 3)                 68483     
Total params: 20,092,867
Trainable params: 67,203
Non-trainable params: 20,025,664
_________________________________________________________________


In [10]:
%%time
for _ in np.arange(10000):
    print(_)
    batch_xs, batch_ys = next_batch(24, train, stacked=True)
    history = pneumonia_vgg19.fit(batch_xs, batch_ys)

0


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch 1/1
24/24 [==============================] - 6s 262ms/step - loss: 1.3075 - categorical_accuracy: 0.4167
1
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 1.2461 - categorical_accuracy: 0.3333
2
Epoch 1/1
24/24 [==============================] - 6s 243ms/step - loss: 1.1479 - categorical_accuracy: 0.3750
3
Epoch 1/1
24/24 [==============================] - 6s 238ms/step - loss: 1.1010 - categorical_accuracy: 0.4583
4
Epoch 1/1
24/24 [==============================] - 6s 258ms/step - loss: 0.8663 - categorical_accuracy: 0.6667
5
Epoch 1/1
24/24 [==============================] - 6s 258ms/step - loss: 1.2722 - categorical_accuracy: 0.5833
6
Epoch 1/1
24/24 [==============================] - 7s 285ms/step - loss: 1.3517 - categorical_accuracy: 0.4583
7
Epoch 1/1
24/24 [==============================] - 6s 265ms/step - loss: 1.1446 - categorical_accuracy: 0.5417
8
Epoch 1/1
24/24 [==============================] - 6s 256ms/step - loss: 1.1337 - categorical_ac

72
Epoch 1/1
24/24 [==============================] - 7s 271ms/step - loss: 0.9797 - categorical_accuracy: 0.5000
73
Epoch 1/1
24/24 [==============================] - 6s 270ms/step - loss: 0.6371 - categorical_accuracy: 0.7083
74
Epoch 1/1
24/24 [==============================] - 6s 267ms/step - loss: 1.0041 - categorical_accuracy: 0.5417
75
Epoch 1/1
24/24 [==============================] - 6s 263ms/step - loss: 1.2557 - categorical_accuracy: 0.4583
76
Epoch 1/1
24/24 [==============================] - 6s 267ms/step - loss: 1.3086 - categorical_accuracy: 0.3333
77
Epoch 1/1
24/24 [==============================] - 6s 260ms/step - loss: 1.1958 - categorical_accuracy: 0.5000
78
Epoch 1/1
24/24 [==============================] - 6s 268ms/step - loss: 1.1902 - categorical_accuracy: 0.5000
79
Epoch 1/1
24/24 [==============================] - 7s 287ms/step - loss: 0.8133 - categorical_accuracy: 0.5833
80
Epoch 1/1
24/24 [==============================] - 7s 291ms/step - loss: 0.7573 - cat

144
Epoch 1/1
24/24 [==============================] - 7s 283ms/step - loss: 1.0697 - categorical_accuracy: 0.4167
145
Epoch 1/1
24/24 [==============================] - 6s 266ms/step - loss: 0.6772 - categorical_accuracy: 0.7500
146
Epoch 1/1
24/24 [==============================] - 6s 269ms/step - loss: 0.6446 - categorical_accuracy: 0.7917
147
Epoch 1/1
24/24 [==============================] - 7s 273ms/step - loss: 0.9852 - categorical_accuracy: 0.5000
148
Epoch 1/1
24/24 [==============================] - 6s 261ms/step - loss: 0.8285 - categorical_accuracy: 0.6667
149
Epoch 1/1
24/24 [==============================] - 7s 280ms/step - loss: 0.9913 - categorical_accuracy: 0.5833
150
Epoch 1/1
24/24 [==============================] - 6s 265ms/step - loss: 1.0355 - categorical_accuracy: 0.5833
151
Epoch 1/1
24/24 [==============================] - 6s 268ms/step - loss: 0.9363 - categorical_accuracy: 0.5833
152
Epoch 1/1
24/24 [==============================] - 6s 261ms/step - loss: 0.9

...

9670
Epoch 1/1
24/24 [==============================] - 6s 259ms/step - loss: 0.7982 - categorical_accuracy: 0.7083
9671
Epoch 1/1
24/24 [==============================] - 6s 261ms/step - loss: 0.6297 - categorical_accuracy: 0.7500
9672
Epoch 1/1
24/24 [==============================] - 6s 264ms/step - loss: 0.9550 - categorical_accuracy: 0.5417
9673
Epoch 1/1
24/24 [==============================] - 6s 256ms/step - loss: 0.5258 - categorical_accuracy: 0.8333
9674
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.8548 - categorical_accuracy: 0.4583
9675
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.7219 - categorical_accuracy: 0.7917
9676
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 1.0810 - categorical_accuracy: 0.5833
9677
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.7286 - categorical_accuracy: 0.7083
9678
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - 

9741
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.9046 - categorical_accuracy: 0.6250
9742
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.8158 - categorical_accuracy: 0.5000
9743
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.9368 - categorical_accuracy: 0.5833
9744
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.6928 - categorical_accuracy: 0.7083
9745
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.8939 - categorical_accuracy: 0.4583
9746
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.6638 - categorical_accuracy: 0.7917
9747
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.7299 - categorical_accuracy: 0.6667
9748
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.6381 - categorical_accuracy: 0.7500
9749
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - 

9812
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.7327 - categorical_accuracy: 0.6667
9813
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.7492 - categorical_accuracy: 0.5000
9814
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.6950 - categorical_accuracy: 0.7500
9815
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.6056 - categorical_accuracy: 0.7917
9816
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.8503 - categorical_accuracy: 0.5000
9817
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.7779 - categorical_accuracy: 0.5833
9818
Epoch 1/1
24/24 [==============================] - 6s 263ms/step - loss: 0.8369 - categorical_accuracy: 0.7500
9819
Epoch 1/1
24/24 [==============================] - 6s 262ms/step - loss: 0.7346 - categorical_accuracy: 0.7500
9820
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - 

9883
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.9390 - categorical_accuracy: 0.5000
9884
Epoch 1/1
24/24 [==============================] - 6s 251ms/step - loss: 0.7751 - categorical_accuracy: 0.7500
9885
Epoch 1/1
24/24 [==============================] - 6s 256ms/step - loss: 0.6776 - categorical_accuracy: 0.6667
9886
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.8084 - categorical_accuracy: 0.5417
9887
Epoch 1/1
24/24 [==============================] - 6s 258ms/step - loss: 0.5797 - categorical_accuracy: 0.8750
9888
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.7516 - categorical_accuracy: 0.6667
9889
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.5882 - categorical_accuracy: 0.7083
9890
Epoch 1/1
24/24 [==============================] - 6s 254ms/step - loss: 0.7036 - categorical_accuracy: 0.5833
9891
Epoch 1/1
24/24 [==============================] - 6s 259ms/step - 

9954
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.6402 - categorical_accuracy: 0.7083
9955
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.7599 - categorical_accuracy: 0.6667
9956
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - loss: 0.6574 - categorical_accuracy: 0.7917
9957
Epoch 1/1
24/24 [==============================] - 6s 252ms/step - loss: 0.5979 - categorical_accuracy: 0.6667
9958
Epoch 1/1
24/24 [==============================] - 6s 253ms/step - loss: 0.9875 - categorical_accuracy: 0.4167
9959
Epoch 1/1
24/24 [==============================] - 6s 262ms/step - loss: 0.6060 - categorical_accuracy: 0.6667
9960
Epoch 1/1
24/24 [==============================] - 6s 259ms/step - loss: 0.7700 - categorical_accuracy: 0.7083
9961
Epoch 1/1
24/24 [==============================] - 6s 259ms/step - loss: 0.8820 - categorical_accuracy: 0.5000
9962
Epoch 1/1
24/24 [==============================] - 6s 255ms/step - 

In [11]:
#Predicting on the test set
#Empty list for prediction classes
prediction = []
#Class of actual values
actual = []
#Going through dataset
for i in np.arange(250):
    #starting index
    start = i * 24
    #End index
    end = (i + 1) * 24
    #Creating the batch dataset
    temp_data = test[start:end]
    #Cetting batch
    x_test, y_test = next_batch(24, temp_data, stacked=True, shuffle=False)
    #Putting the actual class in the actual list
    actual.extend(np.argmax(y_test, axis = 1))
    #Putting the predicted class in the predicted 
    prediction.extend(pneumonia_vgg19.predict_classes(x_test))

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [12]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(actual, prediction)

array([[ 788,  708,  504],
       [ 380, 1460,  160],
       [ 284,  104, 1612]])

In [13]:
print(classification_report(actual, prediction, 
        target_names = ["No Lung Opacity / Not Normal", "Lung Opacity", "Normal"]))

                              precision    recall  f1-score   support

No Lung Opacity / Not Normal       0.54      0.39      0.46      2000
                Lung Opacity       0.64      0.73      0.68      2000
                      Normal       0.71      0.81      0.75      2000

                 avg / total       0.63      0.64      0.63      6000



In [14]:
from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(y_pred=prediction, y_true=actual)
print("Accuracy on test set: {:.3f}".format(test_accuracy))

Accuracy on test set: 0.643


In [15]:
pneumonia_vgg19.save_weights("vgg19_classifier_weights.h5")
pneumonia_vgg19.save('vgg19_classifier.h5')
with open("vgg19_pneumonia_architecture.json", "w") as f:
    f.write(pneumonia_vgg19.to_json())